In [ ]:


# Import python packages
import streamlit as st
import pandas as pd  #%%
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session

session = get_active_session()



     

In [ ]:

sales = session.table("SALES").to_pandas()
purchase = session.table("PURCHASES").to_pandas()
invoice = session.table("INVOICE_PURCHASE").to_pandas()
end_inv = session.table("END_INV").to_pandas()
beg_inv = session.table("BEG_INV").to_pandas()
display(beg_inv.head())
display(end_inv.head())


In [ ]:

nuniques = {"beg_inv": beg_inv.nunique(), "end_inv": end_inv.nunique()}
display(
    pd.DataFrame(nuniques).T[
        [
            "INVENTORYID",
            "STORE",
            "CITY",
            "BRAND",
            "DESCRIPTION",
            "SIZE",
            "ONHAND",
            "BEGINDATE",
            "ENDDATE",
        ]
    ]
)
print(
    f"BEG_INV BRAND nunique: {beg_inv.BRAND.nunique()}, desc + size nunique: {(beg_inv['DESCRIPTION'] + ' ' + beg_inv['SIZE']).nunique()}??? Might need cleaning"
)
print(
    f"end_inv Brand nunique: {end_inv.BRAND.nunique()}, desc + size nunique: {(end_inv['DESCRIPTION'] + ' ' + end_inv['SIZE']).nunique()}"
)
beg_inv_brand = beg_inv.loc[:]
beg_inv_brand["DESC_SIZE"] = beg_inv_brand["DESCRIPTION"] + " " + beg_inv_brand["SIZE"]
group_desc = (
    beg_inv_brand[["BRAND", "DESC_SIZE"]].groupby("DESC_SIZE")["BRAND"].unique()
)
group_desc.loc[group_desc.apply(len) > 1]

display(sales.head())
display(
    beg_inv.loc[
        (beg_inv["BRAND"] == 1004) & (beg_inv["INVENTORYID"] == "1_HARDERSFIELD_1004")
        ]
)
# ? Inventory ID = store_city_brand, Brand = description + Size, With Inventory ID we can find how many onhand the inventory have at the beginning and end.
print(sales.CLASSIFICATION.unique())

## 1. Aggregate the data from the tables

- **Group** the `sales` DataFrame by the `"SALESDATE"` column.  
- **Aggregate** by summing `"SALESQUANTITY"` to get total daily sales.  
- Result stored in a new DataFrame `sales_quantity_price`.



In [ ]:
# * group by date, sum sales quantity to get total sales quantity per day
sales_quantity_price = sales.groupby("SALES_DATE").agg({"SALES_QUANTITY": "sum"})

sales_quantity_price.describe();

print(sales_quantity_price);



## 2. Native Snowflake ML Forecasting

Snowflake provides built-in time-series forecasting as a SQL object you train and invoke entirely in SQL

In [ ]:
df_to_write = sales_quantity_price.reset_index()


snowpark_df = session.create_dataframe(df_to_write)  
snowpark_df.write.mode("overwrite").save_as_table("SALES_QUANTITY_PRICE");


In [ ]:
-- 1) Train the model (requires CREATE SNOWFLAKE.ML.FORECAST privilege)
CREATE SNOWFLAKE.ML.FORECAST inventory_forecast_model (
  INPUT_DATA       => TABLE(sales_quantity_price),
  TIMESTAMP_COLNAME=> 'SALES_DATE',
  TARGET_COLNAME   => 'SALES_QUANTITY'
);



In [ ]:
-- 2) Generate a 90-day forecast
CREATE or REPLACE table sales_quantity_price_forecast AS
SELECT * 
FROM TABLE(inventory_forecast_model!FORECAST(FORECASTING_PERIODS => 90));

In [ ]:
# ── 2) Load your tables into pandas (using SALES_DATE & SALES_QUANTITY) ─────────
sales_df = (
    session
    .table("SALES")
    .select("SALES_DATE", "SALES_QUANTITY")
    .to_pandas()
)
sq_df = (
    session
    .table("SALES_QUANTITY_PRICE")
    .select("SALES_DATE", "SALES_QUANTITY")
    .to_pandas()
)

# Ensure the date columns are datetime
sales_df["SALES_DATE"] = pd.to_datetime(sales_df["SALES_DATE"])
sq_df["SALES_DATE"]    = pd.to_datetime(sq_df["SALES_DATE"])



In [ ]:

# ── 3) Merge on SALES_DATE ───────────────────────────────────────────────────────
merged = pd.merge(
    sales_df.rename(columns={"SALES_QUANTITY": "Raw_Sales"}),
    sq_df.rename(columns={"SALES_QUANTITY": "Agg_Sales"}),
    on="SALES_DATE",
    how="inner"
)
# Rename for plotting
merged = merged.rename(columns={"SALES_DATE": "Date"}).set_index("Date")


### Summary

- **Built-in FORECAST** only returns future forecasts, not fitted training values :contentReference[oaicite:1]{index=1}.  
- To get in-sample predictions you must either **wrap** an external ARIMA implementation in a Python UDF or run your ARIMA entirely in a Container Runtime notebook and persist the `fittedvalues` yourself.  
- Snowflake does not currently expose a table function for fitted (in-sample) values from `SNOWFLAKE.ML.FORECAST`.  

In [ ]:


# 4) Render with Streamlit
st.title("📈 Raw vs Aggregated Sales Quantity")
st.line_chart(merged.iloc[::32])

# Load historical and forecast tables into Pandas ─────────────────────────

In [ ]:
# ── 4) Build visualization DataFrame ───────────────────────────────────────────
# Focus on forecast and its bounds
viz_df = (
    merged
    .set_index("SALES_DATE")
    .sort_index()[["SALES_QUANTITY", "LOWER_BOUND", "UPPER_BOUND"]]
)

# ── 5) Render in Streamlit ─────────────────────────────────────────────────────
st.title("🕒 Sales Quantity Forecast with Prediction Intervals")
st.line_chart(viz_df)


In [ ]:

sp_df = (
    session
    .table("SALES_QUANTITY_PRICE")
    .select("SALES_DATE", "SALES_QUANTITY")
    .to_pandas()
)


spf_df = (
    session
    .table("SALES_QUANTITY_PRICE_FORECAST")
    .select("TS", "FORECAST", "LOWER_BOUND", "UPPER_BOUND")
    .to_pandas()
)



In [ ]:
# ── Prepare DataFrames ───────────────────────────────────────────────────────
# 1) Parse the historical SALES_DATE column
sp_df["SALES_DATE"] = pd.to_datetime(sp_df["SALES_DATE"])

# 2) Parse the forecast SALES_DATE column
spf_df["SALES_DATE"] = pd.to_datetime(spf_df["SALES_DATE"])

# 3) Merge on SALES_DATE (outer to include all dates)
merged = pd.merge(
    sp_df,
    spf_df,
    on="SALES_DATE",
    how="outer"
)

# 4) For forecast periods, replace missing SALES_QUANTITY with the forecast value
merged["SALES_QUANTITY"] = merged["SALES_QUANTITY"].fillna(merged["FORECAST"])

# 5) (Optional) Drop the now-redundant FORECAST column
merged = merged.drop(columns=["FORECAST"])


## We want to integrate the process into Cortex to see the power of DATA + ML + LLM

In [ ]:
CALL SNOWFLAKE.MODELS.CORTEX_BASE_MODELS_REFRESH();
SHOW MODELS IN SNOWFLAKE.MODELS;

In [ ]:
sp_forecast_df = session.create_dataframe(merged)  
sp_forecast_df.write.mode("overwrite").save_as_table("SALES_FORECAST_FULL");

In [ ]:
CREATE or REPLACE TABLE CORTEX_OUT AS
SELECT
  SNOWFLAKE.CORTEX.COMPLETE(
    'LLAMA3-8B',
    $$  
    I have a table SALES_FORECAST_FULL with columns:
      • SALES_DATE  
      • SALES_QUANTITY (actual or imputed forecast)  
      • FORECAST  
      • LOWER_BOUND  
      • UPPER_BOUND  
      
    Please summarize the key trends in the forecast, comment on how the model performed during the historical period, and highlight any notable patterns or anomalies in the 90-day forecast.  
    $$
  ) AS analysis;

In [ ]:
SELECT * FROM CORTEX_OUT

Based on the provided table, I'll summarize the key trends in the forecast, comment on the model's performance during the historical period, and highlight any notable patterns or anomalies in the 90-day forecast.

**Key Trends:**

1. **Trend in Sales Quantity:** The trend in sales quantity over time can be analyzed by plotting the actual or imputed forecast values against the sales date. This will help identify any upward or downward trends, seasonality, or anomalies.
2. **Forecast Accuracy:** The mean absolute error (MAE) or mean absolute percentage error (MAPE) can be calculated to evaluate the model's performance in predicting sales quantity. A lower MAE or MAPE indicates better forecast accuracy.
3. **Confidence Intervals:** The lower and upper bounds can be used to calculate the confidence intervals for the forecast. This will provide a range of possible values for the sales quantity, giving an idea of the uncertainty associated with the forecast.

**Model Performance during Historical Period:**

1. **Forecast Accuracy:** Analyze the MAE or MAPE for the historical period to evaluate the model's performance. A lower MAE or MAPE indicates better forecast accuracy.
2. **Trend in Forecast Errors:** Plot the forecast errors (actual - forecast) over time to identify any patterns or trends in the errors. This can help identify areas where the model is consistently under- or over-forecasting.
3. **Seasonality:** Examine the sales data and forecast for seasonality, which can be identified by plotting the data over time and looking for repeating patterns.

**Notable Patterns or Anomalies in 90-day Forecast:**

1. **Anomalies:** Identify any unusual or unexpected patterns in the 90-day forecast, such as sudden changes in sales quantity or unusual spikes in demand.
2. **Trends:** Analyze the 90-day forecast for any emerging trends or patterns that may not be present in the historical data.
3. **Confidence Intervals:** Use the lower and upper bounds to calculate the confidence intervals for the 90-day forecast. This will provide a range of possible values for the sales quantity, giving an idea of the uncertainty associated with the forecast.

To perform these analyses, you can use statistical software such as R or Python, or data visualization tools like Tableau or Power BI. The specific steps and techniques will depend on the complexity of your data and the insights you're trying to gain.

Here's a sample R code to get you started:
```R
# Load necessary libraries
library(ggplot2)
library(forecast)

# Load the data
data <- read.csv("SALES_FORECAST_FULL.csv")

# Plot the sales quantity over time
ggplot(data, aes(x = SALES_DATE, y = SALES_QUANTITY)) + 
  geom_line() + 
  theme_classic()

# Calculate the mean absolute error (MAE)
mae <- mean(abs(data$SALES_QUANTITY - data$FORECAST))

# Calculate the mean absolute percentage error (MAPE)
mape <- mean(abs((data$SALES_QUANTITY - data$FORECAST) / data$SALES_QUANTITY)) * 100

# Plot the forecast errors over time
ggplot(data, aes(x = SALES_DATE, y = actual - forecast)) + 
  geom_line() + 
  theme_classic()

# Calculate the confidence intervals for the 90-day forecast
forecast_90d <- forecast(data, h = 90)
conf_int <- forecast_90d$conf.int

# Plot the 90-day forecast with confidence intervals
ggplot(forecast_90d, aes(x = time, y = fit)) + 
  geom_line() + 
  geom_ribbon(aes(ymin = conf_int[, 1], ymax = conf_int[, 2]), alpha = 0.2) + 
  theme_classic()
```
This code provides a basic example of how to load the data, plot the sales quantity over time, calculate the MAE and MAPE, plot the forecast errors, and calculate the confidence intervals for the 90-day forecast. You can modify and extend this code to suit your specific needs and gain deeper insights into your sales forecast data.